In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import callbacks
from keras import backend as K
from keras.datasets import mnist
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error
%matplotlib inline
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

Using TensorFlow backend.


In [2]:
from keras.datasets import cifar10
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from keras.backend import clear_session

In [4]:
#dataset CIFAR10
#Please apply your own dataset :)
#train
train=cifar10.load_data()[0][0]
train_label=cifar10.load_data()[0][1]
train_label=train_label.reshape(len(train_label))
#test
x_test=cifar10.load_data()[1][0]
y_test=cifar10.load_data()[1][1]
y_test=y_test.reshape(len(y_test))
data3=np.zeros([len(train),32,32,3])
for n in range(len(train)):
    data3[n]=Image.fromarray(train[n].astype("uint8"))
train=data3

In [5]:
#image size
M=train.shape[1]
N=train.shape[2]
C=train.shape[3]

In [3]:
#The path of goal image. Please use your own goal image :)
images=["Lenna.png"]
#images=["image1.png","image2.png","image3.png"]

In [6]:
clear_session()

In [ ]:
convo=[0,1,2,3,4,5,6,7,8,13,18]
table=[]
for goal in images:
    #goal image I, equation(3)
    I=Image.open(goal).resize([M,N])
    AUC_list=[]
    # Q is corresponding to Fig.6.
    for Q in convo:
        AUC=[]
        # e is random_state. We use 5 different random seed.
        for e in range(1):
            AUC_score=np.zeros(10)
            clear_session()
            for i in range(10):
                x_train=train[train_label==i]
                I_train=np.zeros([len(x_train),M,N,C])
                for n in range(len(x_train)):
                    I_train[n]=I
                I_train=I_train

                x_test=x_test.reshape(len(x_test),M,N,C)
                y_label=np.zeros(len(y_test))
                y_label[y_test==i]=1
                # バックエンドに依存したチャネルの位置を調整する
                if K.image_data_format() == 'channels_last':
                    x_train = x_train.reshape(x_train.shape[0],
                                              M, N,C)
                    #x_test = x_test.reshape(x_test.shape[0],image_height, image_width, 1)
                    I_train = I_train.reshape(I_train.shape[0],
                                              M, N,C)
                    input_shape = (M, N,C)
                else:
                    x_train = x_train.reshape(x_train.shape[0],
                                              C, M, N)
                    x_test = x_test.reshape(x_test.shape[0],C, M, N)
                    input_shape = (C, M, N)

                x_train = x_train.astype('float32')
                I_train = I_train.astype('float32')
                x_train, x_valid, I_train, I_valid = train_test_split(x_train, I_train, test_size=0.3, random_state=e)
                x_valid, x_threshold, I_valid, I_threshold = train_test_split(x_valid, I_valid, test_size=0.3, random_state=e)
                #x_test = x_test.astype('float32')
                #x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
                #x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())

                # model f:X → I
                model = models.Sequential()
                # Repeat convolution layer Q times
                model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',input_shape=input_shape))
                for n in range(Q):
                    model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
                model.add(layers.Conv2D(C, kernel_size=(3, 3),activation='relu', padding='same'))
                # 28 x 28 x 1
                model.compile(optimizer='adam',
                              loss='mean_absolute_error')
                fit_callbacks = [
                    callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5,
                                            mode='min')
                ]
                # Train model
                model.fit(x_train, I_train,
                          epochs=100,
                          batch_size=16,
                          shuffle=True,
                          validation_data=(x_valid, I_valid),callbacks=fit_callbacks)
                #f_threshold = ITN output from x_threshold
                f_threshold=model.predict(x_threshold)
                f_test=model.predict(x_test)
                I_train=I_train.reshape(I_train.shape[0],int(M*N) ,C)
                goal_image=I_train[0]
                f_threshold=f_threshold.reshape(f_threshold.shape[0],int(M*N),C)
                f_test=f_test.reshape(f_test.shape[0],int(M*N),C)
                # equation (12)
                Eth=np.zeros(len(f_threshold))
                for n in range(len(f_threshold)):
                    Eth[n]=mean_absolute_error(goal_image,f_threshold[n])
                Etest=np.zeros(len(f_test))
                for n in range(len(f_test)):
                    Etest[n]=mean_absolute_error(goal_image,f_test[n])
                m=0
                #multiply construction error by (-1)
                AUC_score[i]=roc_auc_score(y_label,(-1)*Etest)               
            AUC.append(AUC_score)    
        AUC_list.append(np.array(AUC).mean())
    table.append(AUC_list)

In [14]:
pd.DataFrame(table).T

,0
0,0.652844
1,0.690771
2,0.712400
3,0.733174
4,0.754190
5,0.752625
6,0.759927
7,0.741707
8,0.713078
9,0.531717
